In [2]:
import pickle
import numpy as np
import os
class DataManager(object):
    '''data manager'''
    def __init__(self, data_path, need_shuffle=False):
        self.data_path = data_path
        self.all_data = []
        self.all_labels = []
        self.need_shuffle = need_shuffle
        self.indicate = 0
    
    def data_process(self):
        self.data_load()
        self.filter_binary_data()
        if self.need_shuffle:
            self.shuffle_data()
        return self.all_data, self.all_labels
        
    def data_load(self):
        print("loading data ......")
        for file in self.data_path:
            print("file:", file)
            if os.path.isfile(file):
                with open(file, "rb") as f:
                    loaded_data = pickle.load(f, encoding="ISO-8859-1")
                    data = loaded_data.get("data")
                    label = loaded_data.get("labels")
                    self.all_data.append(data)
                    self.all_labels.append(label)
        self.all_data = np.vstack(self.all_data)
        self.all_labels = np.hstack(self.all_labels)
        self.all_data = self.all_data/127.5
        print(self.all_data.shape)
        print(self.all_labels.shape)
        print("first 3 data:", self.all_data[0: 3])
        print("first 10 label:", self.all_labels[0: 10])

    def filter_binary_data(self):
        print("fiter_binary_data......")
        binary_data = []
        binary_labels = []
        for item, label in zip(self.all_data, self.all_labels):
            if label in [0, 1]:
                binary_data.append(item)
                binary_labels.append(label)
#         print("binary data shape", binary_data.shape)
#         print("binary label shape", binary_labels)
        print("first 3 data:", self.all_data[0: 3])
        print("first 10 label:", self.all_labels[0: 10])
        self.all_data = np.asarray(binary_data)
        self.all_labels = np.asarray(binary_labels)
        print("binary data shape", self.all_data.shape)
        print("binary label shape", self.all_labels)
    
    def shuffle_data(self):
#         print("shuffle_data.......")
        idx = np.random.permutation(self.all_data.shape[0])
        self.all_data = self.all_data[idx]
        self.all_labels = self.all_labels[idx]
#         print("first 3 data:", self.all_data[0: 3])
#         print("first 3 label:", self.all_labels[0: 10])
    
    def next_batch_data(self, batch_size):
        if batch_size > self.all_data.shape[0]:
            raise Exception("batch size is larger than data size")
        if self.indicate + batch_size > self.all_data.shape[0]:
            if self.need_shuffle:
                self.shuffle_data()
                self.indicate = 0
            else:
                raise Exception("has no more data")
        batch_data = self.all_data[self.indicate: self.indicate + batch_size]
        batch_label = self.all_labels[self.indicate: self.indicate + batch_size]
        self.indicate = self.indicate + batch_size
        return batch_data, batch_label

cifar_dir = "../data/cifar-10-batches-py"
# cifar_dir = "workspace/workspace_py/learn/deepLearning/data/cifar-10-batches-py"
train_data_file = [cifar_dir + "/data_batch_%d" % i for i in range(1, 6)]
test_data_file = [cifar_dir + "/test_batch"]

train_data_manager = DataManager(train_data_file, need_shuffle=True)
train_data_manager.data_process()

test_data_manager = DataManager(test_data_file, need_shuffle=False)
test_data_manager.data_process()
# data_manager = DataManager(test_data_file)
# data_manager.data_load()
# data_manager.filter_binary_data()
# data_manager.shuffle_data()


loading data ......
file: ../data/cifar-10-batches-py/data_batch_1
file: ../data/cifar-10-batches-py/data_batch_2
file: ../data/cifar-10-batches-py/data_batch_3
file: ../data/cifar-10-batches-py/data_batch_4
file: ../data/cifar-10-batches-py/data_batch_5
(50000, 3072)
(50000,)
first 3 data: [[0.4627451  0.3372549  0.39215686 ... 1.09803922 0.65882353 0.56470588]
 [1.20784314 0.98823529 0.82352941 ... 1.09019608 1.11372549 1.12941176]
 [2.         1.98431373 1.98431373 ... 0.65098039 0.65098039 0.65882353]]
first 10 label: [6 9 9 4 1 1 2 7 8 3]
fiter_binary_data......
first 3 data: [[0.4627451  0.3372549  0.39215686 ... 1.09803922 0.65882353 0.56470588]
 [1.20784314 0.98823529 0.82352941 ... 1.09019608 1.11372549 1.12941176]
 [2.         1.98431373 1.98431373 ... 0.65098039 0.65098039 0.65882353]]
first 10 label: [6 9 9 4 1 1 2 7 8 3]
binary data shape (10000, 3072)
binary label shape [1 1 0 ... 0 1 1]
loading data ......
file: ../data/cifar-10-batches-py/test_batch
(10000, 3072)
(10000

(array([[1.21568627, 1.30980392, 1.38039216, ..., 0.39215686, 0.40784314,
         0.39215686],
        [1.25490196, 1.45098039, 1.63921569, ..., 0.24313725, 0.26666667,
         0.21960784],
        [1.70196078, 1.64705882, 1.60784314, ..., 1.6       , 1.76470588,
         1.74901961],
        ...,
        [1.21568627, 1.19215686, 1.09019608, ..., 0.81568627, 0.90196078,
         0.94117647],
        [0.89411765, 0.90196078, 0.90980392, ..., 0.59607843, 0.36078431,
         0.04705882],
        [0.19607843, 0.11764706, 0.18039216, ..., 0.62745098, 0.63529412,
         0.62745098]]), array([0, 1, 1, ..., 0, 0, 1]))

In [1]:

import tensorflow as tf

x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])

'''use one output node to classify binary data'''
'''
# w = tf.get_variable("w", [x.get_shape()[-1], 1], initializer=tf.random_normal_initializer(0, 1))
# b = tf.get_variable("b", [1], initializer=tf.constant_initializer(0.0))



y_ = tf.matmul(x, w) + b
p_y_1 = tf.nn.sigmoid(y_)

y_shaped = tf.reshape(y,  (-1, 1))

loss = tf.reduce_mean(tf.square(tf.cast(y_shaped, tf.float32) - p_y_1))

pred_y = p_y_1 > 0.5
pred_y_int = tf.cast(pred_y, tf.int64)
pred_y_equal = tf.equal(pred_y_int, y_shaped)
accuracy = tf.reduce_mean(tf.cast(pred_y_equal, tf.float32))
'''

'''use softmax method to classify binary data'''
w = tf.get_variable("w", [x.get_shape()[-1], 2], initializer=tf.random_normal_initializer(0, 1))
b = tf.get_variable("b", [2], initializer=tf.constant_initializer(0.0))

y_ = tf.matmul(x, w) + b
p_y = tf.nn.softmax(y_)

y_one_hot = tf.one_hot(y, 2, dtype=tf.float32)
loss = tf.reduce_mean(tf.square(y_one_hot - p_y))

pred_y = tf.argmax(p_y, 1)
y_shaped = tf.reshape(y, (-1, 1))
accuracy = tf.reduce_mean(tf.cast(tf.equal(y_shaped, pred_y), tf.float32))


with tf.name_scope("train_op"):
    train_op = tf.train.AdadeltaOptimizer(1e-3).minimize(loss)
    



/home/acedar/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
cifar_dir = "../data/cifar-10-batches-py"
# cifar_dir = "workspace/workspace_py/learn/deepLearning/data/cifar-10-batches-py"
train_data_file = [cifar_dir + "/data_batch_%d" % i for i in range(1, 6)]
test_data_file = [cifar_dir + "/test_batch"]



init_op = tf.global_variables_initializer()


batch_size = 32
iteration = 10000
with tf.Session() as sess:
    sess.run(init_op)
    for i in range(iteration):
        batch_data, batch_label = train_data_manager.next_batch_data(batch_size)
#         batch_data = batch_data.astype('float32')
#         batch_label = batch_label.astype('int64')
#         print("batch_data:", batch_data[0:3])
#         print("batch_label:", batch_label)
        acc_res, loss_res, _ = sess.run([accuracy, loss, train_op], feed_dict={x: batch_data, y: batch_label})
        if (i + 1) % 500 == 0:
            print("train step:%d, loss: %4.5f, acc:%4.5f" % (i + 1 , loss_res, acc_res))


train step:500, loss: 0.65625, acc:0.34375
train step:1000, loss: 0.43750, acc:0.56250
train step:1500, loss: 0.50000, acc:0.50000
train step:2000, loss: 0.59375, acc:0.40625
train step:2500, loss: 0.50000, acc:0.50000
train step:3000, loss: 0.56250, acc:0.43750
train step:3500, loss: 0.46875, acc:0.53125
train step:4000, loss: 0.43750, acc:0.56250
train step:4500, loss: 0.46875, acc:0.53125
train step:5000, loss: 0.50000, acc:0.50000
train step:5500, loss: 0.62500, acc:0.37500
train step:6000, loss: 0.56250, acc:0.43750
train step:6500, loss: 0.59375, acc:0.40625
train step:7000, loss: 0.46875, acc:0.53125
train step:7500, loss: 0.53125, acc:0.46875
train step:8000, loss: 0.50054, acc:0.50000
train step:8500, loss: 0.53125, acc:0.46875
train step:9000, loss: 0.56250, acc:0.43750
train step:9500, loss: 0.50000, acc:0.50000
train step:10000, loss: 0.43750, acc:0.58789
